# 1 Leiðarval hjá Strætó
Í þessu verkefni á að búa til leitarvél fyrir Strætó, notast verður við raungögn frá Strætó
sem er að finna á
https://straeto.is/um-straeto/opin-rafraen-gogn.
Til einföldunar munuð þið nota skrána gtfs.zip sem er að finna á Canvas síðu námskeiðsins til að tryggja að
allir noti sömu útgáfu.


## 2.1 Inntak
Inntakið er tímatafla strætó, listi yfir leiðir, vagna og stoppistöðvar. Við skilgreinum netið
G á eftirfarandi hátt. Hver einasta stoppistöð verður hnútur.
Ef vagn ekur á milli stoppistöðva x og y á tímum t1 og t2 með engum öðrum stoppum á
milli þá búum við til nýjan hnút v með leggina (x, v), (v, y) og merkjum þá með tímunum t_1 og t_2. Hnúturinn v geymir upplýsingar um hvaða vagni sá hnútur tilheyrir, athugið þessi
hnútur verður ekki endurnýttur.
Fyrir leiðina sem er sýnd á myndinni þá samsvarar hún fimm hnútum. Hnútar 1, 3, 5
eru stoppistöðvarnar Hlemmur B, Sólfarið og Harpa. Hnútur 2 tengir saman Hlemm B og
Sólfarið og leggirnir hafa tímann 10:22 og 10:25. Hnútur 4 tengir saman Sólfarið og Hörpu,
fyrri leggurinn hefur tímann 10:25 (ekki sýnilegt á mynd) og sá seinni 10:26. Þetta er dæmi
um löglegan veg þar sem allir leggir samsvara vögnum sem við getum tekið og tímarnir
stemma, þ.e. eru í vaxandi röð

## 2.2 Reiknirit
Einfaldasta tilfellið sem við viljum leysa þá fá um við par af stoppistöðvum og tíma, (x, y, t)
og úttakið er leið í strætókerfinu frá x til y sem fer frá x á tíma t′ ≥ t og lágmarkar
komutímann á áfangastað. Ef við skiptum um strætisvagn þá samsvarar það því að "bíða"á
hnútnum sem er á stoppistöð og taka legg sem tilheyrir öðrum vagni heldur en þeim sem
við komum með.
Til að leysa þetta verkefni er hægt að nota reiknirit Dijkstra með einni breytingu, í stað
þess að slaka (RELAX aðferðin) á leggjum með því að leggja saman vigtir þá notum við
komutímann á leggnum ef leggurinn er löglegur, þ.e. ef (u, v) er leggurinn sem við skoðum
með tímann t þá setjum við d[v] = min(d[v], t) ef t ≥ d[u].


# 2.3 Verkþættir
## 2.3.1 Þáttun (⋆)
Gögnin fyrir strætó eru gefin á GTFS formi, lýsingu á GTFS er að finna á https://gtfs.
org/schedule/reference/. Til að búa til netið þarf að þátta (e. parse) gögnin, þ.e. lesa
þau inn og tengja saman á skynsamlegan hátt.

Til einföldunar ætlum við eingöngu að nota eftirfarandi hluta af gögnunum

agency_id=1 eingöngu leiðir sem tilheyra Strætó BS merktar ST.*

Leiðirnar eru ólíkar milli vikudaga, gefum okkur að við séum að vinna með virka daga
og sleppum næturstrætó (leiðir 101-106). Það má líka sleppa helgidögum, þ.e. sem er
skilgreint í calendar_dates.csv skráni.

Sumar leiðir eru í pöntunarþjónustu, við höldum eingöngu þeim leggjum sem eru með
pickup_type=0 í stop_times.csv.

#### import

In [2]:
import pandas as pd
from datetime import datetime, timedelta
import heapq

#### Hjálparföll

In [12]:
# fall til að finna út hvort við séum með tvö id's með sama nafn
def find_unique_stop_ids(df, stop_name):
    filtered_df = df[df['stop_name'] == stop_name]
    unique_stop_ids = filtered_df['stop_id'].unique()
    return unique_stop_ids

def convert_time_str_to_timedelta(time_str):
    parts = time_str.split(':')
    hours, minutes = int(parts[0]), int(parts[1])
    seconds = int(parts[2]) if len(parts) == 2 else 0
    days, hours = divmod(hours, 24)
    return timedelta(days=days, hours=hours, minutes=minutes, seconds=seconds)

def convert_time_str_to_timedelta_2(time_str):
    # Assuming time_str is in the format "HH:MM"
    hours, minutes = map(int, time_str.split(':'))
    return timedelta(hours=hours, minutes=minutes)

def format_hours_to_time(hours):
    if hours is None:
        return None
    total_minutes = int(hours * 60)
    h, m = divmod(total_minutes, 60)
    return f"{h:02d}:{m:02d}"

def format_hours_to_time_2(timedelta_obj):
    total_seconds = int(timedelta_obj.total_seconds())
    hours, remainder = divmod(total_seconds, 3600)
    minutes, _ = divmod(remainder, 60)
    return f"{hours:02d}:{minutes:02d}"

In [30]:
# Lesa inn gögnin
stops_df = pd.read_csv('content/stops.txt')
trips_df = pd.read_csv('content/trips.txt')
stop_times_df = pd.read_csv('content/stop_times.txt')
routes_df = pd.read_csv('content//routes.txt', comment='#')
calendar_dates_df = pd.read_csv('content/calendar_dates.txt')
# næturstrætó
night_buses = ['101', '102', '103', '104', '105', '106']

# Filter -> bara sem byrjar á ST. & agency_id = 1 & er ekki næturstrætó
routes_df = routes_df[(routes_df['route_id'].str.startswith("ST.", na=False)) &
                      (routes_df['agency_id'] == 1) &
                      (~routes_df['route_short_name'].isin(night_buses))]

# Bæta við dálki sem hefur aðeins upplýsingar um vikudaga.
trips_df['day_pattern'] = trips_df['service_id'].apply(lambda x: x.split('_')[1])

# Filter -> bara virkir dagar.
def all_weekdays_no_weekends(pattern):
    weekdays = 'MTWTF'
    return all(day in pattern for day in weekdays if day != '-')
    
filtered_trips_df = trips_df[trips_df['day_pattern'].apply(all_weekdays_no_weekends)]

# Filter -> pickup_type = 0
filtered_stop_times_df = stop_times_df[stop_times_df['pickup_type'] == 0]

# Merge -> filtered_stop_times & stops til að fá nafn á stoppi
stop_times_with_stops_df = pd.merge(filtered_stop_times_df, stops_df, on='stop_id')

# Merge -> stop_times_with_stops & trips til að fá upplýsingar um ferðir
stop_times_with_stops_and_trips_df = pd.merge(stop_times_with_stops_df, filtered_trips_df, on='trip_id')

# Merge -> Henda þessu í öllu í eina klessu
stop_times_with_stops_trips_and_routes_df = pd.merge(stop_times_with_stops_and_trips_df, routes_df[['route_id', 'route_short_name']], on='route_id')

# Merge -> stop_times_with_stops_trips_and_routes & calendar_dates til að fá upplýsingar um dagsetningar
filtered_final_df = pd.merge(stop_times_with_stops_trips_and_routes_df, calendar_dates_df, on='service_id')

# Filter -> bara fyrir daginn 20240411
filtered_final_current_df = filtered_final_df[filtered_final_df['date'] == 20240411] # finna betri leið service id er unique og nýjasta

# Filter -> óþarfa gögn
df_re_filtered = filtered_final_current_df.drop(['stop_headsign', 'trip_short_name', 'exception_type', 'service_id', 'route_id', 'shape_id', 'date', 'day_pattern', 'pickup_type','location_type'], axis=1)

# raða til að geta sett inn rétta liði
df = df_re_filtered.sort_values(['trip_id', 'stop_sequence'])

# Add 'next_stop_id' by shifting 'stop_id' up by one within each 'trip_id' group
df['next_stop_id'] = df.groupby('trip_id')['stop_id'].shift(-1)
df['next_arrival_time'] = df.groupby('trip_id')['arrival_time'].shift(-1)

# setja flögg til að geta brugðist betur við.
df['is_first_stop'] = df.groupby('trip_id')['stop_sequence'].transform('min') == df['stop_sequence']
df['is_last_stop'] = df.groupby('trip_id')['stop_sequence'].transform('max') == df['stop_sequence']

df['next_stop_id'] = df.groupby('trip_id')['stop_id'].shift(-1)
df['next_arrival_time'] = df.groupby('trip_id')['arrival_time'].shift(-1)

df['next_stop_id'].fillna('No next stop', inplace=True)
df['next_arrival_time'].fillna('End of trip', inplace=True)

filtered_df = df[df['arrival_time'] != df['departure_time']]

display(filtered_df)

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_name,stop_lat,stop_lon,trip_headsign,direction_id,block_id,route_short_name,next_stop_id,next_arrival_time,is_first_stop,is_last_stop
1176,524766,13:49:00,13:51:00,10010802,13,Hamraborg A,64.111643,-21.908606,Hfj. Skarðshlíð,0,4134_1-G,1,10000854.0,13:51:00,False,False
2013,524766,14:03:00,14:06:00,14001500,22,Fjörður,64.067849,-21.957284,Hfj. Skarðshlíð,0,4134_1-G,1,14001561.0,14:07:00,False,False
4896,524767,13:19:00,13:21:00,10010802,13,Hamraborg A,64.111643,-21.908606,Hfj. Skarðshlíð,0,4124_1-B,1,10000854.0,13:21:00,False,False
5733,524767,13:33:00,13:36:00,14001500,22,Fjörður,64.067849,-21.957284,Hfj. Skarðshlíð,0,4124_1-B,1,14001561.0,13:37:00,False,False
8616,524768,12:49:00,12:51:00,10010802,13,Hamraborg A,64.111643,-21.908606,Hfj. Skarðshlíð,0,4130_1-D,1,10000854.0,12:51:00,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4638435,528684,18:49:00,18:51:00,90000075,18,Mjódd,64.109702,-21.842643,Spöng,1,4423_24-A,24,90000348.0,18:55:00,False,False
4642155,528685,18:19:00,18:21:00,90000075,18,Mjódd,64.109702,-21.842643,Spöng,1,4426_24-B,24,90000348.0,18:25:00,False,False
4645875,528686,21:19:00,21:21:00,90000075,18,Mjódd,64.109702,-21.842643,Spöng,1,4426_24-B,24,90000348.0,21:25:00,False,False
4649595,528691,23:19:00,23:21:00,90000075,18,Mjódd,64.109702,-21.842643,Spöng,1,4423_24-A,24,90000348.0,23:25:00,False,False


Búið til net sem samsvarar leiðarkerfinu á virkum degi. Sýnið fjölda hnúta og leggja í
netinu og finnið þá stoppistöð sem hefur hæstu útgráðuna (hvaða stöð og hver er útgráðan).
Netið sem þið búið til má vera hvernig sem er svo lengi sem það sé "pythonskt"í laginu.
Þ.e. við getum ítrað yfir hnúta í netinu og fyrir hvern hnút getum við ítrað yfir granna þess
hnúts. Útfærslan í Java í Algorithms bókinni uppfyllir þessi skilyrði og eftirfarandi einfaldi
python kóði gerir það líka.

```
G = {1: [2,3], 2: [], 3: [1,2] }
for v in G:
  print(u,':')
    for u in G[u,v]:
      print(u,' -> ',v)
```
Ef við viljum bæta við einhverjum upplýsingum þá er hægt að gera það í uppflettitöflu.
Ekki er leyfilegt að nota pakka fyrir net í python á borð við networkx eða sambærilegt.

#### Búum til hakkatöflu fyrir stop_id svo við getum auðveldlega náð í nafn á stoppustöð

In [82]:
# búum til hakkatöflu fyrir stop_id svo við getum auðveldlega náð í nafn á stoppustöð
all_stop_ids = df['stop_id'].unique()
stop_id_to_name = pd.Series(df.stop_name.values, index=df.stop_id).to_dict()

print(stop_id_to_name[90000397])

Árvað


#### Búum til netið G

In [74]:
G = {}
for _, row in df.iterrows():
    stop_id = row['stop_id']
    arrival_time = convert_time_str_to_timedelta(row['arrival_time'])
    departure_time = convert_time_str_to_timedelta(row['departure_time'])
    
    if not row['is_last_stop']:  # Check if it's not the last stop before processing next_stop_id
        next_stop_id = row['next_stop_id']
        next_arrival_time = convert_time_str_to_timedelta(row['next_arrival_time'])
        travel_time = next_arrival_time - arrival_time
        route_short_name = row['route_short_name']
        trip_headsign = row['trip_headsign']
        stop_lat = row['stop_lat']
        stop_lon = row['stop_lon']

        if stop_id not in G:
            G[stop_id] = []
        G[stop_id].append({
            'next_stop_id': next_stop_id,
            'route_short_name': route_short_name,
            'trip_headsign': trip_headsign,
            'arrival_time': arrival_time,
            'travel_time': travel_time,
            'departure_time': departure_time,
            'next_arrival_time': next_arrival_time,
            'stop_lat': stop_lat,
            'stop_lon': stop_lon,
        })
        
# for node, edges in list(G.items())[:1]:  # Adjust the number as needed to inspect more nodes
#     print(f"Node: {node}, Stop Name: {stop_id_to_name[node]}")  # Print the current node's stop name
#     for edge in edges:
#         print(f"  Edge to {edge['to']} (Stop Name: {stop_id_to_name[edge['to']]})")  # Print the destination node's stop name
#         print(f"    On it's way to {edge['trip_headsign']}")
#         print(f"    Route: {edge['route_short_name']}")
#         print(f"    Travel Time: {edge['travel_time']}")
#         print(f"    Arrival Time: {edge['arrival_time']}")
#         print(f"    Departure Time: {edge['departure_time']}")
#         print(f"    Arrival at next stop: {edge['next_arrival_time']}")
#         print(f"    x: {edge['stop_lat']} y: {edge['stop_lon']}")
#     print()

In [16]:
#### spurning hvort megi gera ráð fyrir báðum stoppum. fram og tilbaka. 
#### ég heyrði að lokatalan ætti að vera í kringum 1000 sem myndi meika sens. 2*538 ca 1000


#reikna útgráðu        
out_degrees = {stop_id: len(neighbors) for stop_id, neighbors in G.items()}

# Finna stoppistöð með hæstu útgráðu
max_out_degree_stop = max(out_degrees, key=out_degrees.get)
max_out_degree = out_degrees[max_out_degree_stop]
max_out_degree_stop_name = filtered_final_current_df[filtered_final_current_df['stop_id'] == max_out_degree_stop].iloc[0]['stop_name']

print(f"Stoppistöð með hæstu útgráðu er {max_out_degree_stop_name} með útgráðu {max_out_degree}.")

Stoppistöð með hæstu útgráðu er Mjódd með útgráðu 538.


# 2.3.2 Leit (⋆⋆)
Útfærið leitarreikniritið á netinu eins og lýst var að ofan. Sýnið niðurstöðu leitarinnar þegar
við viljum komast frá Meistaravöllum (ath. það eru tvær stöðvar með þessu nafni) til FB,
kl 11:30.

In [111]:
def find_route(G, starts, destinations, start_time):
    earliest_arrival_to_destination = None
    route_info = []
    start_time_delta = convert_time_str_to_timedelta_2(start_time)
    destinations_set = set(destinations)
    entry_count = 0

    visited = {}
    last_route = None

    for start in starts:
        pq = [(start_time_delta, entry_count, {'stop': start, 'route': None, 'departure_time': start_time_delta}, [])]
        entry_count += 1
        visited[start] = start_time_delta

        while pq:
            current_time, _, current_stop_info, path = heapq.heappop(pq)
            current_stop = current_stop_info['stop']

            if current_stop in destinations_set:
                if earliest_arrival_to_destination is None or current_time < earliest_arrival_to_destination:
                    earliest_arrival_to_destination = current_time
                    route_info = path + [current_stop_info]
                # route_info.append(current_stop_info)
                continue

            for edge in G.get(current_stop, []):
                next_stop = edge['next_stop_id']
                next_route = edge['route_short_name']
                travel_time_delta = edge['travel_time']
                arrival_time_at_next_stop = current_time + travel_time_delta
                trip_headsign = edge['trip_headsign']

                if next_stop not in visited or arrival_time_at_next_stop < visited[next_stop]:
                    visited[next_stop] = arrival_time_at_next_stop
                    next_stop_info = {
                        'stop': next_stop,
                        'route': next_route,
                        'departure_time': current_time,
                        'next_arrival_time': arrival_time_at_next_stop,
                        'trip_headsign': trip_headsign
                    }
                    heapq.heappush(pq, (arrival_time_at_next_stop, entry_count, next_stop_info, path + [current_stop_info]))
                    entry_count += 1

    if route_info:
        for i, stop_info in enumerate(route_info):
            if stop_info['route'] != last_route:  # Þarf að laga
                next_stop_info = route_info[i + 1] if i < len(route_info) - 1 else stop_info
                print(f"Lagt af stað frá {stop_id_to_name[stop_info['stop']]} með strætó nr {stop_info['route']} á leið til {stop_info['trip_headsign']}. "
                      f"Lagt af stað kl: {format_hours_to_time_2(stop_info['departure_time'])}")
                # print(f"Farið út á {stop_id_to_name[next_stop_info['stop']]} kl "
                      # f"{format_hours_to_time_2(next_stop_info['next_arrival_time'])}")

                last_route = stop_info['route']
    print(f"Kominn á áfangastað kl {format_hours_to_time_2(earliest_arrival_to_destination)} með strætó nr {stop_info['route']} á leið til {stop_info['trip_headsign']}. ")
    return earliest_arrival_to_destination



meistaravellir = find_unique_stop_ids(df, 'Meistaravellir')
fb = find_unique_stop_ids(df, 'FB')

find_route(G, meistaravellir, fb,'11:30')

Lagt af stað frá Grandavegur með strætó nr 15 á leið til Mosfellsbær. Lagt af stað kl: 11:30
Lagt af stað frá Skerjagarðar með strætó nr 12 á leið til Skerjafjörður. Lagt af stað kl: 11:35
Lagt af stað frá Þorragata með strætó nr 15 á leið til Mosfellsbær. Lagt af stað kl: 11:36
Lagt af stað frá Læknagarður með strætó nr 1 á leið til Hfj. Skarðshlíð. Lagt af stað kl: 11:38
Lagt af stað frá Kringlan með strætó nr 3 á leið til Sel/Fell. Lagt af stað kl: 11:42
Lagt af stað frá Mjódd með strætó nr 11 á leið til Mjódd. Lagt af stað kl: 11:49
Lagt af stað frá Stekkjarbakki með strætó nr 2 á leið til Hlemmur. Lagt af stað kl: 11:52
Lagt af stað frá Seljabraut með strætó nr 12 á leið til Mjódd/Ártún. Lagt af stað kl: 11:54
Lagt af stað frá Æsufell með strætó nr 3 á leið til Sel/Fell. Lagt af stað kl: 11:56
Kominn á áfangastað kl 12:00 með strætó nr 3 á leið til Sel/Fell. 


datetime.timedelta(seconds=43200)


## 2.3.3 Framsetning (⋆)
Takið löglega leið og búið til leiðarlýsingu, þ.e. hvaða vagn á að taka klukkan hvað og hvar
á að skipta um stöð, svipað og gert er á myndinni að ofan.

In [ ]:

# class Node:  # Hnutur
#     def __init__(self, name: str):
#         self.name = name  # Name of the stop

# class Edge:  # Leggur
#     def __init__(self, start: Node, end: Node, start_time: str, end_time: str):
#         self.start = start  # Starting stop
#         self.end = end  # Ending stop
#         self.start_time = start_time  # Departure time from the start stop
#         self.end_time = end_time  # Arrival time at the end stop


## 2.3.4 Labb (⋆⋆)
Sumar stoppistöðvar eru nálægt hvor annari en ekki hægt að taka strætó á milli þeirra. Breytið netinu og reikniritinu ykkar til að höndla þetta tilfelli, notið GPS hnitin sem eru gefin í gögnunum og veljið hæfilega fjarlægð sem þið eruð til í að láta fólk labba. T.d. er hægt að labba úr 13 í Suðurveri yfir í 4 á Kringlumýrarbraut en 13 stoppar ekki þar. Sýnið dæmi um leið þar sem komutími styttist við að leyfa labb. Þessi aðferð við að tengja saman stöðvar er ekki fullkomin og sýnir að oft er mikilvægara að hafa aðgang að betri gögnum, t.d. eru stoppistöðvar nálægt í planinu en taka langan tíma að labba á milli.

## 2.3.5 Tímamælingar (⋆)
Mælið fjölda fyrirspurna sem aðferðin ykkar getur svarað á sekúndu. Tryggið að mælingarnar blandi stuttum og löngum leiðum með ólíkum tímum.

## 2.3.6 Bestun og gagnagrindur (⋆⋆)
Notið niðurstöður úr tímamælingu til að ná betri afköstum. Hér skiptir máli hvernig framsetningin á netinu er framkvæmd, hvaða gagnagrindur eru notaðar og hvort hægt sé að bæta eitthvað.

## 2.3.7 Færri skiptingar (⋆⋆)
Í einhverjum tilfellum eru margar ólíkar leiðir sem taka jafnlangan tíma, þ.e. seinasti vagninn kemur á áfangastað á sama tíma en fjöldi vagna sem er tekinn er ekki sá sami. Útskýrið hvernig er hægt að breyta reikniritinu til skipta sem sjaldnast um vagn og útfærið þetta á netinu. Sýnið dæmi um strætóferð þar sem niðurstaðan breytist.

## 2.3.8 A(⋆⋆⋆)
Útfærið A∗ reikniritið (sjá kafla 10.8 í https://people.mpi-inf.mpg.de/~mehlhorn/ftp/
NewToolbox/spath.pdf) með því að velja heppilegt fjarlægðarfall f . Sýnið að fjarlægðar-
fallið sem er valið sé löglegt, þ.e. reikniritið skilar alltaf réttum niðurstöðum, og framkvæmið
tímamælingar sem mæla hversu miklu þessi breyting skilar.

## 2.3.9 Pokémon (⋆ ⋆ ⋆)
Finnið þá leið sem tekur mestan fjölda vagna. Athugið að þetta verkefni er mögulega NP-erfitt, takið eftir að netið eins og það er sett upp er ekki DAG (stefnt óhringað net) en það má breyta því í DAG með því að taka út hnútana sem samsvara stoppistöðvum og bæta við hnút fyrir stoppistöð og hvern tímapunkt (x, t), þá setjum við legg frá (x, t) til (x, t′) ef t′ er næsti tímapunktur fyrir þessa stöppistöð og legg sem fara frá (x, t) til v ef vagn fer frá x á tíma t.

Finnið, með einhverju móti, lengstu mögulegu leið sem tekur sem flesta ólíka strætis-
vagnaleiðir.